In [0]:
-- =====================================================
-- DAY 6 POC - CTEs & RECURSIVE QUERIES
-- =====================================================

USE bank_management;

-- =====================================================
-- TOPIC 1: BASIC CTE
-- =====================================================

-- Example: Simple CTE for readability
SELECT '=== Basic CTE Example ===' AS example;

WITH high_balance_accounts AS (
    SELECT 
        account_id,
        customer_id,
        account_number,
        balance
    FROM accounts
    WHERE balance > 10000
)
SELECT 
    c.first_name || ' ' || c.last_name AS customer_name,
    h.account_number,
    h.balance
FROM high_balance_accounts h
JOIN customers c ON h.customer_id = c.customer_id
ORDER BY h.balance DESC;

-- =====================================================
-- TOPIC 2: MULTIPLE CTEs (CHAINED)
-- =====================================================

SELECT '=== Multiple CTEs - Step by Step Analysis ===' AS example;

WITH 
-- Step 1: Calculate per-customer totals
customer_totals AS (
    SELECT 
        c.customer_id,
        c.first_name || ' ' || c.last_name AS customer_name,
        c.customer_type,
        COUNT(a.account_id) AS account_count,
        COALESCE(SUM(a.balance), 0) AS total_balance
    FROM customers c
    LEFT JOIN accounts a ON c.customer_id = a.customer_id
    GROUP BY c.customer_id, c.first_name, c.last_name, c.customer_type
),
-- Step 2: Calculate averages by customer type
type_averages AS (
    SELECT 
        customer_type,
        ROUND(AVG(total_balance), 2) AS avg_balance_for_type
    FROM customer_totals
    GROUP BY customer_type
),
-- Step 3: Compare each customer to their type's average
customer_comparison AS (
    SELECT 
        ct.customer_name,
        ct.customer_type,
        ct.total_balance,
        ta.avg_balance_for_type,
        ROUND(ct.total_balance - ta.avg_balance_for_type, 2) AS difference_from_avg,
        CASE 
            WHEN ct.total_balance > ta.avg_balance_for_type THEN 'Above Average'
            WHEN ct.total_balance < ta.avg_balance_for_type THEN 'Below Average'
            ELSE 'Average'
        END AS performance
    FROM customer_totals ct
    JOIN type_averages ta ON ct.customer_type = ta.customer_type
)
SELECT * FROM customer_comparison
ORDER BY total_balance DESC
LIMIT 10;

-- =====================================================
-- TOPIC 3: REUSING CTE MULTIPLE TIMES
-- =====================================================

SELECT '=== Reusing Same CTE ===' AS example;

WITH monthly_summary AS (
    SELECT 
        DATE_TRUNC('month', transaction_date) AS month,
        COUNT(*) AS transaction_count,
        SUM(amount) AS total_amount,
        AVG(amount) AS avg_amount
    FROM transactions
    GROUP BY DATE_TRUNC('month', transaction_date)
)
-- Use same CTE twice
SELECT 'Top Month' AS metric, month, total_amount
FROM monthly_summary
ORDER BY total_amount DESC
LIMIT 1
UNION ALL
SELECT 'Bottom Month' AS metric, month, total_amount
FROM monthly_summary
ORDER BY total_amount ASC
LIMIT 1;

-- =====================================================
-- TOPIC 4: RECURSIVE CTE - SETUP DATA
-- =====================================================

-- Create hierarchy table for demo
CREATE TABLE IF NOT EXISTS employee_hierarchy (
    employee_id INT,
    employee_name STRING,
    manager_id INT,
    title STRING
);

-- Clear and insert sample data
DELETE FROM employee_hierarchy WHERE employee_id IS NOT NULL;

INSERT INTO employee_hierarchy VALUES
(1, 'Sarah Chen', NULL, 'CEO'),
(2, 'Michael Scott', 1, 'VP Operations'),
(3, 'Angela Martin', 1, 'VP Finance'),
(4, 'Jim Halpert', 2, 'Director Sales'),
(5, 'Pam Beesly', 2, 'Director Marketing'),
(6, 'Dwight Schrute', 4, 'Sales Manager'),
(7, 'Stanley Hudson', 4, 'Sales Manager'),
(8, 'Kevin Malone', 3, 'Accountant'),
(9, 'Oscar Martinez', 3, 'Senior Accountant'),
(10, 'Ryan Howard', 6, 'Sales Rep'),
(11, 'Kelly Kapoor', 5, 'Marketing Specialist');

-- =====================================================
-- TOPIC 5: RECURSIVE CTE - FULL ORG CHART
-- =====================================================

SELECT '=== Recursive CTE - Full Organization Chart ===' AS example;

WITH RECURSIVE org_chart AS (
    -- Anchor: Start with CEO (no manager)
    SELECT 
        employee_id,
        employee_name,
        manager_id,
        title,
        1 AS level,
        CAST(employee_name AS STRING) AS path
    FROM employee_hierarchy
    WHERE manager_id IS NULL
    
    UNION ALL
    
    -- Recursive: Get employees who report to people already in result
    SELECT 
        e.employee_id,
        e.employee_name,
        e.manager_id,
        e.title,
        oc.level + 1,
        CAST(oc.path || ' → ' || e.employee_name AS STRING) AS path
    FROM employee_hierarchy e
    JOIN org_chart oc ON e.manager_id = oc.employee_id
)
SELECT 
    REPEAT('  ', level - 1) || employee_name AS org_structure,
    title,
    level,
    path
FROM org_chart
ORDER BY level, employee_name;

-- =====================================================
-- TOPIC 6: RECURSIVE CTE - FIND ALL REPORTS
-- =====================================================

SELECT '=== Find All Employees Under VP Operations ===' AS example;

WITH RECURSIVE reports AS (
    -- Anchor: Start with VP Operations
    SELECT 
        employee_id,
        employee_name,
        manager_id,
        title,
        0 AS levels_below
    FROM employee_hierarchy
    WHERE employee_name = 'Michael Scott'
    
    UNION ALL
    
    -- Recursive: Get all their direct and indirect reports
    SELECT 
        e.employee_id,
        e.employee_name,
        e.manager_id,
        e.title,
        r.levels_below + 1
    FROM employee_hierarchy e
    JOIN reports r ON e.manager_id = r.employee_id
)
SELECT 
    employee_name,
    title,
    levels_below,
    CASE levels_below
        WHEN 0 THEN 'Manager'
        WHEN 1 THEN 'Direct Report'
        ELSE 'Indirect Report'
    END AS relationship
FROM reports
ORDER BY levels_below, employee_name;

-- =====================================================
-- TOPIC 7: RECURSIVE CTE - MANAGEMENT CHAIN
-- =====================================================

SELECT '=== Management Chain for Sales Rep ===' AS example;

WITH RECURSIVE management_chain AS (
    -- Anchor: Start with the employee
    SELECT 
        employee_id,
        employee_name,
        manager_id,
        title,
        0 AS level_up
    FROM employee_hierarchy
    WHERE employee_name = 'Ryan Howard'
    
    UNION ALL
    
    -- Recursive: Get their manager, then their manager's manager, etc.
    SELECT 
        e.employee_id,
        e.employee_name,
        e.manager_id,
        e.title,
        mc.level_up + 1
    FROM employee_hierarchy e
    JOIN management_chain mc ON e.employee_id = mc.manager_id
)
SELECT 
    level_up,
    employee_name,
    title,
    CASE level_up
        WHEN 0 THEN 'Employee'
        WHEN 1 THEN 'Direct Manager'
        WHEN 2 THEN 'Manager\'s Manager'
        ELSE 'Upper Management'
    END AS relationship
FROM management_chain
ORDER BY level_up;

-- =====================================================
-- TOPIC 8: CTE WITH WINDOW FUNCTIONS
-- =====================================================

SELECT '=== CTE + Window Functions Combined ===' AS example;

WITH transaction_ranking AS (
    SELECT 
        t.transaction_id,
        t.account_id,
        a.account_number,
        t.amount,
        t.transaction_date,
        ROW_NUMBER() OVER (PARTITION BY t.account_id ORDER BY t.amount DESC) AS rank_by_amount,
        SUM(t.amount) OVER (PARTITION BY t.account_id ORDER BY t.transaction_date) AS running_total
    FROM transactions t
    JOIN accounts a ON t.account_id = a.account_id
    WHERE t.transaction_type IN ('Deposit', 'Withdrawal')
)
SELECT 
    account_number,
    transaction_date,
    amount,
    rank_by_amount,
    ROUND(running_total, 2) AS running_total
FROM transaction_ranking
WHERE rank_by_amount <= 3
ORDER BY account_number, rank_by_amount;

-- =====================================================
-- QUESTION 1: TOP N PER CATEGORY WITH CTE
-- =====================================================

SELECT '=== Top 2 Transactions Per Account ===' AS example;

WITH ranked_transactions AS (
    SELECT 
        a.account_number,
        t.transaction_type,
        t.amount,
        t.transaction_date,
        ROW_NUMBER() OVER (
            PARTITION BY a.account_id 
            ORDER BY t.amount DESC
        ) AS rank
    FROM transactions t
    JOIN accounts a ON t.account_id = a.account_id
)
SELECT 
    account_number,
    transaction_type,
    amount,
    transaction_date
FROM ranked_transactions
WHERE rank <= 2
ORDER BY account_number, rank;

-- =====================================================
-- QUESTION 2: COMPLEX AGGREGATION
-- =====================================================

SELECT '=== Customer Lifetime Value Analysis ===' AS example;

WITH 
customer_transactions AS (
    SELECT 
        c.customer_id,
        c.first_name || ' ' || c.last_name AS customer_name,
        COUNT(t.transaction_id) AS total_transactions,
        SUM(CASE WHEN t.transaction_type = 'Deposit' THEN t.amount ELSE 0 END) AS total_deposits,
        SUM(CASE WHEN t.transaction_type = 'Withdrawal' THEN t.amount ELSE 0 END) AS total_withdrawals,
        MIN(t.transaction_date) AS first_transaction,
        MAX(t.transaction_date) AS last_transaction
    FROM customers c
    JOIN accounts a ON c.customer_id = a.customer_id
    JOIN transactions t ON a.account_id = t.account_id
    GROUP BY c.customer_id, c.first_name, c.last_name
),
customer_metrics AS (
    SELECT 
        *,
        total_deposits - total_withdrawals AS net_flow,
        DATEDIFF(last_transaction, first_transaction) AS days_active,
        ROUND(total_transactions * 1.0 / NULLIF(DATEDIFF(last_transaction, first_transaction), 0), 2) AS transactions_per_day
    FROM customer_transactions
)
SELECT 
    customer_name,
    total_transactions,
    ROUND(total_deposits, 2) AS total_deposits,
    ROUND(total_withdrawals, 2) AS total_withdrawals,
    ROUND(net_flow, 2) AS net_flow,
    days_active,
    transactions_per_day,
    CASE 
        WHEN net_flow > 10000 THEN 'High Value'
        WHEN net_flow > 5000 THEN 'Medium Value'
        ELSE 'Standard'
    END AS customer_segment
FROM customer_metrics
ORDER BY net_flow DESC;

-- =====================================================
-- QUESTION 3: HIERARCHICAL ROLLUP
-- =====================================================

SELECT '=== Department Headcount Rollup ===' AS example;

WITH RECURSIVE dept_rollup AS (
    -- Base: Count direct reports
    SELECT 
        manager_id AS employee_id,
        COUNT(*) AS direct_reports,
        0 AS total_team_size
    FROM employee_hierarchy
    WHERE manager_id IS NOT NULL
    GROUP BY manager_id
    
    UNION ALL
    
    -- Recursive: Add up team sizes up the chain
    SELECT 
        e.manager_id,
        COUNT(DISTINCT e.employee_id) AS direct_reports,
        SUM(dr.direct_reports + dr.total_team_size) AS total_team_size
    FROM employee_hierarchy e
    JOIN dept_rollup dr ON e.employee_id = dr.employee_id
    WHERE e.manager_id IS NOT NULL
    GROUP BY e.manager_id
)
SELECT 
    e.employee_name,
    e.title,
    COALESCE(dr.direct_reports, 0) AS direct_reports,
    COALESCE(dr.total_team_size, 0) AS total_team_size
FROM employee_hierarchy e
LEFT JOIN dept_rollup dr ON e.employee_id = dr.employee_id
WHERE e.manager_id IS NULL OR dr.direct_reports > 0
ORDER BY total_team_size DESC;

-- =====================================================
-- COMPARISON: SUBQUERY vs CTE
-- =====================================================

SELECT '=== Same Query: Subquery vs CTE ===' AS example;

-- Subquery approach (harder to read)
SELECT 
    customer_name,
    total_balance
FROM (
    SELECT 
        c.first_name || ' ' || c.last_name AS customer_name,
        SUM(a.balance) AS total_balance
    FROM customers c
    JOIN accounts a ON c.customer_id = a.customer_id
    GROUP BY c.customer_id, c.first_name, c.last_name
) AS subquery_result
WHERE total_balance > 20000
ORDER BY total_balance DESC
LIMIT 3;

-- CTE approach (more readable)
WITH customer_balances AS (
    SELECT 
        c.first_name || ' ' || c.last_name AS customer_name,
        SUM(a.balance) AS total_balance
    FROM customers c
    JOIN accounts a ON c.customer_id = a.customer_id
    GROUP BY c.customer_id, c.first_name, c.last_name
)
SELECT customer_name, total_balance
FROM customer_balances
WHERE total_balance > 20000
ORDER BY total_balance DESC
LIMIT 3;

-- =====================================================
-- SUMMARY
-- =====================================================

SELECT '=== DAY 6 CONCEPTS SUMMARY ===' AS summary;
SELECT 
    'Basic CTE' AS concept,
    'WITH name AS (SELECT...)' AS syntax,
    'Improves readability' AS benefit
UNION ALL
SELECT 'Multiple CTEs', 'WITH cte1 AS (...), cte2 AS (...)', 'Chain logical steps'
UNION ALL
SELECT 'Recursive CTE', 'WITH RECURSIVE ... UNION ALL', 'Query hierarchies'
UNION ALL
SELECT 'CTE Reuse', 'Reference same CTE twice', 'Avoid repeating logic'
UNION ALL
SELECT 'CTE + Windows', 'Combine for complex analysis', 'Powerful combination'
UNION ALL
SELECT 'Anchor + Recursive', 'Base case + iteration', 'Build hierarchy level by level';

-- Clean up
DROP TABLE IF EXISTS employee_hierarchy;

-- =====================================================
-- END OF DAY 6 POC
-- =====================================================

---------------------------------------------------------------------------
ParseException                            Traceback (most recent call last)
File <command-7671861032519022>, line 1
----> 1 get_ipython().run_cell_magic('sql', '', "-- =====================================================\n-- DAY 6 POC - CTEs & RECURSIVE QUERIES\n-- =====================================================\n\nUSE bank_management;\n\n-- =====================================================\n-- TOPIC 1: BASIC CTE\n-- =====================================================\n\n-- Example: Simple CTE for readability\nSELECT '=== Basic CTE Example ===' AS example;\n\nWITH high_balance_accounts AS (\n    SELECT \n        account_id,\n        customer_id,\n        account_number,\n        balance\n    FROM accounts\n    WHERE balance > 10000\n)\nSELECT \n    c.first_name || ' ' || c.last_name AS customer_name,\n    h.account_number,\n    h.balance\nFROM high_balance_accounts h\nJOIN customers c ON h.customer